In [4]:
import boto3
import pandas as pd
import json

## todos
- add the other networks
- add logic to pull the actual token transactions (but Alchemy not The Graph)

In [12]:
with open('test.json', 'r') as f:
    test_string = f.read()


In [15]:
json_data = json.loads(test_string)

## Alright, first I need to get the locks table

1. Who deployed the lock
- lock_managers
2. The lock metadata
- `address`
- `name`
- `price`
- `blockNumber` = some sort of created at?
3. Who has the lock (keys)


In [22]:
json_data['locks'][1]

{'address': '0x06c04c79b3d2e56d697f4c13e25aa130061e261c',
 'name': 'CRYPTO GIRLS COMMUNITY',
 'blockNumber': '16106074',
 'price': '0',
 'expirationDuration': '115792089237316195423570985008687907853269984665640564039457584007913129639935',
 'lockManagers': ['0x952580d41f10db41d97fcd6b1984bc2538eefc2c'],
 'keys': [{'id': '0x06c04c79b3d2e56d697f4c13e25aa130061e261c-1',
   'owner': '0x952580d41f10db41d97fcd6b1984bc2538eefc2c',
   'expiration': '115792089237316195423570985008687907853269984665640564039457584007913129639935',
   'tokenURI': 'https://locksmith.unlock-protocol.com/api/key/0x06c04c79b3d2e56d697f4c13e25aa130061e261c/1'}]}

In [59]:
## dropping the init value
locks_df = pd.DataFrame(json_data['locks'][1])
locks_df = locks_df.drop(columns=['keys','lockManagers'])
locks_df.head()

,address,name,blockNumber,price,expirationDuration
0,0x06c04c79b3d2e56d697f4c13e25aa130061e261c,CRYPTO GIRLS COMMUNITY,16106074,0,1157920892373161954235709850086879078532699846...


In [81]:
## lock managers
lock_managers_df = pd.DataFrame(json_data['locks'][1:])


In [82]:
lock_managers_df_filtered = lock_managers_df.filter(['address', 'lockManagers']) 


In [83]:
lock_managers_df_filtered.head()

,address,lockManagers
0,0x06c04c79b3d2e56d697f4c13e25aa130061e261c,[0x952580d41f10db41d97fcd6b1984bc2538eefc2c]
1,0xa45b36f2a425037ccb6799efb33ea13dab348c97,[0xd8fe942c2bd17c3b73892eee55934308e9463e62]
2,0x0486e122199c021325b7bbfeca90a33884e293d7,[0x4d7e84e5643df4862ce2e1280c0bcf9ffba4285a]
3,0x388e600b5b3b75f59a433765e3c0492394ee5655,[0x4d7e84e5643df4862ce2e1280c0bcf9ffba4285a]
4,0xbd9f72486fb0b967d7b24225b5a98551f03d8d94,[0xc75446a6adaef73269dbdece73536977b2b639e0]


In [85]:
lock_managers_df_filtered.explode('lockManagers', inplace=True)


TypeError: explode() got an unexpected keyword argument 'inplace'

In [88]:
lock_managers_df_filtered = lock_managers_df_filtered.explode('lockManagers')
lock_managers_df_filtered = lock_managers_df_filtered.rename(columns={'lockManagers': 'lockManager'})
lock_managers_df_filtered.head()

,address,lockManager
0,0x06c04c79b3d2e56d697f4c13e25aa130061e261c,0x952580d41f10db41d97fcd6b1984bc2538eefc2c
1,0xa45b36f2a425037ccb6799efb33ea13dab348c97,0xd8fe942c2bd17c3b73892eee55934308e9463e62
2,0x0486e122199c021325b7bbfeca90a33884e293d7,0x4d7e84e5643df4862ce2e1280c0bcf9ffba4285a
3,0x388e600b5b3b75f59a433765e3c0492394ee5655,0x4d7e84e5643df4862ce2e1280c0bcf9ffba4285a
4,0xbd9f72486fb0b967d7b24225b5a98551f03d8d94,0xc75446a6adaef73269dbdece73536977b2b639e0


In [97]:
## okay keys
keys = pd.DataFrame(json_data['keys'][1:])
keys.head()

,lockAddress,keyId,owner,expiration,tokenURI
0,0x06c04c79b3d2e56d697f4c13e25aa130061e261c,0x06c04c79b3d2e56d697f4c13e25aa130061e261c-1,0x952580d41f10db41d97fcd6b1984bc2538eefc2c,1157920892373161954235709850086879078532699846...,https://locksmith.unlock-protocol.com/api/key/...
1,0xbd9f72486fb0b967d7b24225b5a98551f03d8d94,0xbd9f72486fb0b967d7b24225b5a98551f03d8d94-1,0x81dd955d02d337db81ba6c9c5f6213e647672052,1672440167,https://locksmith.unlock-protocol.com/api/key/...
2,0xc5bfc92057bbd051c1701faececb7a8c69b78913,0xc5bfc92057bbd051c1701faececb7a8c69b78913-1,0x86f2ad57b59bb5be8091a0a5fdbecb168b63ca17,1157920892373161954235709850086879078532699846...,https://locksmith.unlock-protocol.com/api/key/...
3,0xc5bfc92057bbd051c1701faececb7a8c69b78913,0xc5bfc92057bbd051c1701faececb7a8c69b78913-10,0x4de5be3cffcef257717fa89e7aa763055202b4b6,1672455059,https://kroniclabz.mypinata.cloud/ipfs/QmZSWat...
4,0xc5bfc92057bbd051c1701faececb7a8c69b78913,0xc5bfc92057bbd051c1701faececb7a8c69b78913-11,0xfa10a0c9885b66f4bff4c419b58db33aefa6e840,1672462091,https://kroniclabz.mypinata.cloud/ipfs/QmZSWat...


In [96]:
key_metadata = keys.filter(['lockAddress', 'keyId', 'expiration', 'tokenUri'])
key_metadata.head()

,lockAddress,keyId,expiration
0,0x06c04c79b3d2e56d697f4c13e25aa130061e261c,0x06c04c79b3d2e56d697f4c13e25aa130061e261c-1,1157920892373161954235709850086879078532699846...
1,0xbd9f72486fb0b967d7b24225b5a98551f03d8d94,0xbd9f72486fb0b967d7b24225b5a98551f03d8d94-1,1672440167
2,0xc5bfc92057bbd051c1701faececb7a8c69b78913,0xc5bfc92057bbd051c1701faececb7a8c69b78913-1,1157920892373161954235709850086879078532699846...
3,0xc5bfc92057bbd051c1701faececb7a8c69b78913,0xc5bfc92057bbd051c1701faececb7a8c69b78913-10,1672455059
4,0xc5bfc92057bbd051c1701faececb7a8c69b78913,0xc5bfc92057bbd051c1701faececb7a8c69b78913-11,1672462091


In [101]:
## okay key holders
key_holders = pd.DataFrame(json_data['keys'][1:])
key_holders = keys.filter(['owner', 'lockAddress', 'keyId'])

    

In [102]:
key_holders['tokenId'] = key_holders['keyId'].apply(lambda x: x.split('-')[1])

In [103]:
key_holders.head()

,owner,lockAddress,keyId,tokenId
0,0x952580d41f10db41d97fcd6b1984bc2538eefc2c,0x06c04c79b3d2e56d697f4c13e25aa130061e261c,0x06c04c79b3d2e56d697f4c13e25aa130061e261c-1,1
1,0x81dd955d02d337db81ba6c9c5f6213e647672052,0xbd9f72486fb0b967d7b24225b5a98551f03d8d94,0xbd9f72486fb0b967d7b24225b5a98551f03d8d94-1,1
2,0x86f2ad57b59bb5be8091a0a5fdbecb168b63ca17,0xc5bfc92057bbd051c1701faececb7a8c69b78913,0xc5bfc92057bbd051c1701faececb7a8c69b78913-1,1
3,0x4de5be3cffcef257717fa89e7aa763055202b4b6,0xc5bfc92057bbd051c1701faececb7a8c69b78913,0xc5bfc92057bbd051c1701faececb7a8c69b78913-10,10
4,0xfa10a0c9885b66f4bff4c419b58db33aefa6e840,0xc5bfc92057bbd051c1701faececb7a8c69b78913,0xc5bfc92057bbd051c1701faececb7a8c69b78913-11,11


In [113]:
import chunk
import boto3
import os
import logging
import json
import boto3
from botocore.exceptions import ClientError
import pandas as pd


class S3Utils:
    def __init__(self):
        self.s3_client = boto3.client("s3")
        self.s3_resource = boto3.resource("s3")

    def save_json(self, bucket_name, filename, data):
        "This will save the data field to the S3 bucket set during initialization. The data must be a JSON compliant python object."
        try:
            content = bytes(json.dumps(data).encode("UTF-8"))
        except Exception as e:
            logging.error("The data does not seem to be JSON compliant.")
            raise e
        try:
            self.s3_client.put_object(Bucket=bucket_name, Key=filename, Body=content)
        except Exception as e:
            logging.error("Something went wrong while uploading to S3!")
            raise e

    def save_file(self, bucket_name, local_path, s3_path):
        "This will save the data field to the S3 bucket set during initialization. The data must be a JSON compliant python object."
        try:
            self.s3_client.upload_file(local_path, bucket_name, s3_path)
        except Exception as e:
            logging.error("Something went wrong while uploading to S3!")
            raise e

    def save_json_as_csv(self, data, bucket_name, file_name, ACL="public-read"):
        """Function to save a python list of dictionaries (json compatible) to a CSV in S3.
        This functions takes care of splitting the array if the resulting CSV is more than 10Mb.
        parameters:
        - data: the data array.
        - bucket_name: The bucket name
        - file_name: The file name (without .csv at the end)
        - ACL: (Optional) defaults to public-read for neo4J ingestion."""
        df = pd.DataFrame.from_dict(data)
        chunks = [df]
        # Check if the dataframe is bigger than the max allowed size of Neo4J (10Mb)
        if df.memory_usage(index=False).sum() > 10000000:
            chunks = self.split_dataframe(df)

        urls = []
        for chunk, chunk_id in zip(chunks, range(len(chunks))):
            chunk.to_csv(f"s3://{bucket_name}/{file_name}--{chunk_id}.csv", index=False)
            self.s3_resource.ObjectAcl(bucket_name, f"{file_name}--{chunk_id}.csv").put(ACL=ACL)
            location = self.s3_client.get_bucket_location(Bucket=bucket_name)["LocationConstraint"]
            urls.append("https://s3-%s.amazonaws.com/%s/%s" % (location, bucket_name, f"{file_name}--{chunk_id}.csv"))
        return urls
    
    def save_df_as_csv(self, df, bucket_name, file_name, ACL="public-read"):
        """Function to save a Pandas DataFrame to a CSV file in S3.
        This functions takes care of splitting the dataframe if the resulting CSV is more than 10Mb.
        parameters:
        - df: the dataframe to be saved.
        - bucket_name: The bucket name.
        - file_name: The file name (without .csv at the end).
        - ACL: (Optional) defaults to public-read for neo4J ingestion."""
        chunks = [df]
        # Check if the dataframe is bigger than the max allowed size of Neo4J (10Mb)
        if df.memory_usage(index=False).sum() > 10000000:
            chunks = self.split_dataframe(df)

        urls = []
        for chunk, chunk_id in zip(chunks, range(len(chunks))):
            chunk.to_csv(f"s3://{bucket_name}/{file_name}--{chunk_id}.csv", index=False)
            self.s3_resource.ObjectAcl(bucket_name, f"{file_name}--{chunk_id}.csv").put(ACL=ACL)
            location = self.s3_client.get_bucket_location(Bucket=bucket_name)["LocationConstraint"]
            urls.append("https://s3-%s.amazonaws.com/%s/%s" % (location, bucket_name, f"{file_name}--{chunk_id}.csv"))
        return urls


    def load_csv(self, bucket_name, file_name):
        """Convenience function to retrieve a S3 saved CSV loaded as a pandas dataframe."""
        df = pd.read_csv(f"s3://{bucket_name}/{file_name}", lineterminator="\n")
        return df

    # def set_object_private(BUCKET, file_name, resource):
    #     object_acl = resource.ObjectAcl(BUCKET, file_name)
    #     response = object_acl.put(ACL="private")

    def split_dataframe(self, df, chunk_size=10000):
        chunks = list()
        num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size else 0)
        for i in range(num_chunks):
            chunks.append(df[i * chunk_size : (i + 1) * chunk_size])
        return chunks

    def load_json(self, bucket_name, filename):
        "Retrieves a JSON formated content from the S3 bucket"
        try:
            result = self.s3_client.get_object(Bucket=bucket_name, Key=filename)
        except Exception as e:
            logging.error("An error occured while retrieving data from S3!")
            raise e
        data = json.loads(result["Body"].read().decode("UTF-8"))
        return data

    def check_if_file_exists(self, bucket_name, filename):
        "This checks if the filename to be saved already exists and raises an error if so."
        try:
            boto3.resource("s3").Object(bucket_name, filename).load()
        except ClientError as e:
            if e.response["Error"]["Code"] != "404":
                logging.error("Something went wrong while checking if the data file already existed in the bucket!")
                raise e
            else:
                return False
        else:
            return True

    def create_or_get_bucket(self, bucket_name):
        response = self.s3_client.list_buckets()
        if bucket_name not in [el["Name"] for el in response["Buckets"]]:
            try:
                logging.warning("Bucket not found! Creating {}".format(bucket_name))
                location = {"LocationConstraint": os.environ["AWS_DEFAULT_REGION"]}
                self.s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
                logging.info(f"Creating bucket: {bucket_name}")
            except ClientError as e:
                logging.error("An error occured during the creation of the bucket!")
                raise e
        else:
            logging.info(f"Using existing bucket: {bucket_name}")
        return boto3.resource("s3").Bucket(bucket_name)



In [116]:
s3_utils = S3Utils()


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [120]:
!pip install fsspec s3fs
s3_utils.save_df_as_csv(lockDf, 'unlock_test', 'lockMetadata')

  Using cached aiobotocore-2.1.2.tar.gz (58 kB)
  Preparing metadata (setup.py) ... done
  Using cached botocore-1.23.24-py3-none-any.whl (8.4 MB)
  Using cached aioitertools-0.11.0-py3-none-any.whl (23 kB)
Using legacy 'setup.py install' for aiobotocore, since package 'wheel' is not installed.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.26.10
    Uninstalling botocore-1.26.10:
      Successfully uninstalled botocore-1.26.10
    Running setup.py install for aiobotocore ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.23.10 requires botocore<1.27.0,>=1.26.10, but you have botocore 1.23.24 which is incompatible.


PermissionError: Anonymous access is forbidden for this operation

In [38]:
lockDf.head()

,address,name,blockNumber,price,expirationDuration,lockManagers
0,0x06c04c79b3d2e56d697f4c13e25aa130061e261c,CRYPTO GIRLS COMMUNITY,16106074,0,1157920892373161954235709850086879078532699846...,[0x952580d41f10db41d97fcd6b1984bc2538eefc2c]
1,0xa45b36f2a425037ccb6799efb33ea13dab348c97,Ronan Hunt Lock,16104780,0,1157920892373161954235709850086879078532699846...,[0xd8fe942c2bd17c3b73892eee55934308e9463e62]
2,0x0486e122199c021325b7bbfeca90a33884e293d7,second,16094321,0,1157920892373161954235709850086879078532699846...,[0x4d7e84e5643df4862ce2e1280c0bcf9ffba4285a]
3,0x388e600b5b3b75f59a433765e3c0492394ee5655,second,16094320,0,1157920892373161954235709850086879078532699846...,[0x4d7e84e5643df4862ce2e1280c0bcf9ffba4285a]
4,0xbd9f72486fb0b967d7b24225b5a98551f03d8d94,Chainverse for Community - Early Supporter,16080593,84820000,2592000,[0xc75446a6adaef73269dbdece73536977b2b639e0]


In [ ]:
## i need to get the key transactions for longtiduinal data

{
  keyPurchases(orderBy: timestamp, orderDirection: desc, first: 5) {
    lock
    purchaser
  }
}